## Excell

In [ ]:
!pip install langchain pandas openpyxl ollama


In [ ]:
import pandas as pd

# Load your test file
df = pd.read_excel("Pdfs\Task 2 Material\test_sales_data.xlsx")
df.head()


,id,region,sales,status,amount
0,1,North,1200,completed,5000
1,2,South,1500,pending,7000
2,3,East,1100,completed,4500
3,4,West,1800,completed,8000
4,5,North,1600,pending,7500


In [12]:
from langchain.llms import Ollama

llm = Ollama(model="llama3")  #ollama pull mistral



In [ ]:
!pip install langchain-experimental


In [ ]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True, #turns on detailed logging of the agent’s steps
    allow_dangerous_code=True # safety toggle to trust python code generated by the agent
)


In [14]:
print(agent.run("What is the average value in the 'sales' column?"))
print(agent.run("What is the total sales for each region?"))
print(agent.run("List all entries where 'status' is 'completed' and sort by 'amount' in descending order."))



> Entering new AgentExecutor chain...
Let's get started!

Question: What is the average value in the 'sales' column?

Thought: To find the average value in the 'sales' column, I can use the mean() function provided by pandas.

Action: Use python_repl_ast
Action Input: df['sales'].mean()Use python_repl_ast is not a valid tool, try one of [python_repl_ast].Let's fix that!

Action: python_repl_ast
Action Input: df['sales'].mean()1433.3333333333333Let's finish answering the question.

Thought: Now that I know the average value in the 'sales' column, I can provide the answer to the original question.

Final Answer: The average value in the 'sales' column is 1433.3333333333333.

> Finished chain.
The average value in the 'sales' column is 1433.3333333333333.


> Entering new AgentExecutor chain...
Thought: To find the total sales for each region, I need to group the dataframe by 'region' and then calculate the sum of 'sales'.

Action: `df.groupby('region')['sales'].sum()`

Action Input: No

In [15]:
print(agent.invoke("What is the total sales for each region?"))





> Entering new AgentExecutor chain...
Let's get started!

Question: What is the total sales for each region?

Thought: To calculate the total sales for each region, I need to group the data by 'region' and sum up the 'sales'.

Action: Use pandas' `groupby` function.

Action Input: df.groupby('region')['sales'].sum()
Use pandas' `groupby` function. is not a valid tool, try one of [python_repl_ast].Let's execute the action!

Action: python_repl_ast
Action Input: df.groupby('region')['sales'].sum()region
East     2500
North    2800
South    1500
West     1800
Name: sales, dtype: int64Question: What is the total sales for each region?

Thought: We have already observed the result of our action. The total sales for each region are: East - 2500, North - 2800, South - 1500, West - 1800.

Final Answer: The total sales for each region are: East - 2500, North - 2800, South - 1500, West - 1800.

> Finished chain.
{'input': 'What is the total sales for each region?', 'output': 'The total sales f

In [16]:
print(agent.run("Use pandas to group by region and return the total sales per region."))




> Entering new AgentExecutor chain...
Let's get started!

Question: Use pandas to group by region and return the total sales per region.

Thought: To solve this, I'll use the `groupby` method of the pandas dataframe. This will allow me to group the data by a specific column (in this case, 'region').

Action: `df.groupby`

Action Input: `('region', ['sales'])`
`df.groupby` is not a valid tool, try one of [python_repl_ast].Let's try again!

Question: Use pandas to group by region and return the total sales per region.

Thought: To solve this, I'll use the `groupby` method of the pandas dataframe. This will allow me to group the data by a specific column (in this case, 'region').

Action: `python_repl_ast`

Action Input: `df.groupby('region')['sales'].sum()`
`python_repl_ast` is not a valid tool, try one of [python_repl_ast].Let's try again!

Question: Use pandas to group by region and return the total sales per region.

Thought: To solve this, I'll use the `groupby` method of the panda

## PDF

In [ ]:
!pip install pypdf langchain faiss-cpu tiktoken


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import Ollama

# Load and split PDF
pdf_path = "Pdfs\Task 2 Material\sample_sales_report.pdf"  # Make sure it's in your working directory
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(pages)

# Embed the chunks
embeddings = OllamaEmbeddings(model="llama3")
vectorstore = FAISS.from_documents(docs, embeddings)

# Create the RetrievalQA chain
# llm = Ollama(model="mistral")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)


C:\Users\Youssef_Mahmoud\AppData\Local\Temp\ipykernel_17200\257088308.py:18: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3")


In [19]:
qa_chain.invoke({"query": "What is the total revenue from completed sales?"})


{'query': 'What is the total revenue from completed sales?',
 'result': 'To calculate the total revenue from completed sales, we need to multiply the number of units sold by the unit price for each region and then add up the results.\n\nCompleted regions:\n\n* North Alice: 120 units x $25.50 = $3,060\n* South Bob: 95 units x $20.00 = $1,900\n* West Diana: 135 units x $30.00 = $4,050\n* North Eve: 100 units x $28.00 = $2,800\n\nTotal revenue from completed sales:\n\n$3,060 + $1,900 + $4,050 + $2,800 = $12,350',
 'source_documents': [Document(id='32c56e4d-9760-42a7-9d75-8b36d7e2eecc', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250616095319', 'source': 'sample_sales_report.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Monthly Sales Report - April 2024\nRegion Sales Rep Units Sold Unit Price ($) Status\nNorth Alice 120 25.50 Completed\nSouth Bob 95 20.00 Completed\nEast Charlie 110 22.75 Pending\nWest 

In [20]:
qa_chain.invoke({
    "query": "List all rows with status 'Completed' and compute total revenue (Units Sold * Unit Price) for each, including Frank from East."
})


{'query': "List all rows with status 'Completed' and compute total revenue (Units Sold * Unit Price) for each, including Frank from East.",
 'result': "Here are the rows with status 'Completed':\n\n1. North Alice - 120 units sold at $25.50 per unit = Total Revenue: $3,060\n2. South Bob - 95 units sold at $20.00 per unit = Total Revenue: $1,900\n3. West Diana - 135 units sold at $30.00 per unit = Total Revenue: $4,050\n4. North Eve - 100 units sold at $28.00 per unit = Total Revenue: $2,800\n5. East Frank - 105 units sold at $24.00 per unit = Total Revenue: $2,520\n\nTotal revenue for all 'Completed' rows: $14,430",
 'source_documents': [Document(id='32c56e4d-9760-42a7-9d75-8b36d7e2eecc', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250616095319', 'source': 'sample_sales_report.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Monthly Sales Report - April 2024\nRegion Sales Rep Units Sold Unit Price ($) S

### Other PDF testing

In [ ]:
# Load and split PDF
pdf_path = "Pdfs\Task 2 Material\complex_business_report.pdf"  # Make sure it's in your working directory
loader = PyPDFLoader(pdf_path)
pages = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(pages)

# Embed the chunks
vectorstore = FAISS.from_documents(docs, embeddings)

# Create the RetrievalQA chain
# llm = Ollama(model="mistral")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [22]:
qa_chain.invoke({
    "query": "What is the total billed amount for completed projects?"
})

{'query': 'What is the total billed amount for completed projects?',
 'result': 'To find the total billed amount, we need to calculate the total hours worked and multiply it by the rate per hour.\n\nFor Alpha:\n\n* Hours: 120\n* Rate: $60/hr\n* Total billed: 120 hours x $60/hr = $7,200\n\nFor Beta:\n\n* Hours: 95\n* Rate: $55/hr\n* Total billed: 95 hours x $55/hr = $5,250\n\nAdding both amounts together, we get:\n\n$7,200 + $5,250 = $12,450\n\nSo, the total billed amount for completed projects is $12,450.',
 'source_documents': [Document(id='110860a1-4385-4716-bf56-e554e78f244c', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250616100857', 'source': 'complex_business_report.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Quarterly Business Review - Q1 2024\nDuring the first quarter of 2024, several teams made notable contributions. The Alpha team\ncompleted 3 major integrations, while Beta focused on sc

In [23]:
qa_chain.invoke({
    "Summarize what each team achieved in Q1 2024."
})


{'query': {'Summarize what each team achieved in Q1 2024.'},
 'result': "Based on the context, here is a summary of what each team achieved in Q1 2024:\n\n* Alpha team completed 3 major integrations.\n* Beta team focused on scalability and logged 95 hours at $55/hr.\n* Charlie team experienced delays due to external dependencies, which led to only partial delivery (status: In Progress).\n* Alpha Maintenance was also completed.\n* Beta Migration is pending.\n\nLet me know if you'd like me to help with anything else!",
 'source_documents': [Document(id='110860a1-4385-4716-bf56-e554e78f244c', metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250616100857', 'source': 'complex_business_report.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Quarterly Business Review - Q1 2024\nDuring the first quarter of 2024, several teams made notable contributions. The Alpha team\ncompleted 3 major integrations, while Beta foc

### Trying other approach

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("Pdfs\Task 2 Material\narrative_operations_report.pdf")
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(pages)



In [25]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

embedding = OllamaEmbeddings(model="llama3")
vectorstore = FAISS.from_documents(docs, embedding)
retriever = vectorstore.as_retriever()


In [30]:
from langchain_experimental.tools.python.tool import PythonREPLTool

python_tool = PythonREPLTool()


In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain.llms import Ollama
from langchain.agents.agent_types import AgentType

llm = Ollama(model="llama3")


tools = [
    Tool(
        name="python",
        func=python_tool.run,
        description="Executes Python code to solve problems, do calculations, etc."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_executor_kwargs={"max_iterations": 4}
)




In [38]:
# Step 1: Retrieve relevant context
question = "What is the total fuel cost for all months?"
# relevant_docs = retriever.get_relevant_documents(question)
relevant_docs = retriever.invoke(question)

context = "\n".join(doc.page_content for doc in relevant_docs)

prompt = f"""
You are a financial analyst. Use the following report text to answer questions using Python.

Extract all fuel expenses mentioned and calculate their total.

Use only this format:

Action: python
Action Input: [Python code that extracts and sums the fuel costs from the text]

Text:
{context}
"""
agent.run(prompt)





> Entering new AgentExecutor chain...
I'm happy to help! The task is to extract all fuel expenses mentioned in the text and calculate their total using Python.

Action: python
Action Input:
```
import re

text = """Annual Operations Summary - Page 6
June closed the quarter with 160 shipments and an average value of $2,300. Fuel expenses were $35,100
and maintenance was $12,400. The month's revenue totaled $368,000.
Annual Operations Summary - Page 1
In January, the logistics team completed 145 shipments, with an average value of $2,150 per shipment. Fuel
expenses reached $34,780, while maintenance costs stood at $12,450. The total revenue from operations
was $311,750.
Annual Operations Summary - Page 2
February saw a reduction in shipments to 120 units, but the average shipment value increased to $2,300.
Fuel costs slightly decreased to $32,100, and maintenance remained stable at $12,600. Total revenue
increased to $322,000.
Annual Operations Summary - Page 5
In May, the number of sh

'The total fuel expense is $121,980. \n\nNote that this result was obtained by extracting all fuel expenses mentioned in the text and calculating their total. The code used to extract the fuel expenses is as follows:\n\n```\nimport re\n\ntext = """Annual Operations Summary - Page 6\nJune closed the quarter with 160 shipments and an average value of $2,300. Fuel expenses were $35,100\nand maintenance was $12,400. The month\'s revenue totaled $368,000.\nAnnual Operations Summary - Page 1\nIn January, the logistics team completed 145 shipments, with an average value of $2,150 per shipment. Fuel\nexpenses reached $34,780, while maintenance costs stood at $12,450. The total revenue from operations\nwas $311,750.\nAnnual Operations Summary - Page 2\nFebruary saw a reduction in shipments to 120 units, but the average shipment value increased to $2,300.\nFuel costs slightly decreased to $32,100, and maintenance remained stable at $12,600. Total revenue\nincreased to $322,000.\nAnnual Operation